![logo](../images/logo-poster.png)

In [ ]:
%run supportvectors-common.ipynb

# Neural Network for classification of the breast cancer dataset

We will extend on the simple neural architecture to now solve a classification problem

In [2]:
from svlearn.approximator.regression_network import SimpleFeedForwardNet, \
                                                    SimpleNumpyDataset

import numpy as np
import torch

from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader

## Batch Gradient Descent

We will use the breast cancer dataset for building this neural network classification model

## Homework 

Study the preprocess_data to understand the transformations being done on the ingested raw data

In [3]:
import pandas as pd
from svlearn.breast_cancer.pre_process import preprocess_data
from svlearn.breast_cancer.ingest_data import ingest_breast_cancer_data

In [5]:
data = ingest_breast_cancer_data()
preprocessed_data = preprocess_data(data)
x = preprocessed_data.drop(['target'], axis=1).to_numpy(dtype=np.float32)
y = preprocessed_data[['target']].to_numpy(dtype=np.float32)

In [6]:
dim_x = x.shape[1]
X_tensor = torch.from_numpy(x).reshape(-1, dim_x)
y_tensor = torch.from_numpy(y)

In [ ]:
x.shape

In [ ]:
# Get the training parts ready
network = SimpleFeedForwardNet(input_dimension=dim_x, output_dimension=1)
print(network)
network.activation = torch.relu # Rectified Linear Unit
loss_function = BCEWithLogitsLoss()
optimizer = Adam(network.parameters(), lr=0.01)

# Let us now train the network
losses = []
epochs = 2001
drop_out = 0.1
for epoch in range(epochs):

    optimizer.zero_grad()  # reset the gradients
    results = network(X_tensor, drop_out)  # get predictions
    loss = loss_function(results, y_tensor)  # estimate loss
    loss.backward()  # back-propagate gradients
    optimizer.step()  # update the parameter values (gradient-descent)
    losses.append(loss.data)  # keep track of the loss of this epoch
    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.data))

### Plot the results of batch gradient descent on the data

In [ ]:
from svlearn.approximator.regression_network import create_plots

create_plots(epochs, losses)

## Mini-batch Gradient Descent

Here, at each step, we learn from from mini batches of data. So, for this, the dataloader returns data in small batches. The mini-batch size is specified in the pytorch dataloader by the parameter `batch_size`

In [ ]:
dataset = SimpleNumpyDataset(x, y)
loader = DataLoader(dataset, batch_size=50, shuffle=True)


epochs = 101
drop_out = 0.1
# Get the training parts ready
network = SimpleFeedForwardNet(input_dimension=dim_x, output_dimension=1)
print(network)
loss_function = BCEWithLogitsLoss()
optimizer = Adam(network.parameters(), lr=0.001)
losses = []
steps = 0
for epoch in range(epochs):
    start = True
    for data, labels in loader:
        optimizer.zero_grad()  # reset the parameter gradients
        results = network(data, drop_out)  # get predictions
        loss = loss_function(results, labels)  # estimate loss
        loss.backward()  # back-propagate gradients
        optimizer.step()  # update the parameter values (gradient-descent)
        losses.append(loss.data)  # keep track of the loss of this epoch
        if epoch % 10 == 0 and start:
            print('epoch {}, loss {}'.format(epoch, loss.data))
        start = False
        steps +=1

### Plot the results of MINI-BATCH gradient descent on the data

In [ ]:
create_plots(steps, losses)

## Stochastic Gradient Descent

Here, each step of learning is from one datum. So an epoch will have as many steps as the training sample size. Let us see how well it does.

In [ ]:
# Note how we have set the mini-batch size to 1!
loader = DataLoader(dataset, batch_size=1, shuffle=True)
epochs = 50
drop_out = 0.1
# Get the training parts ready
network = SimpleFeedForwardNet(input_dimension=dim_x, output_dimension=1)

print(network)
loss_function = BCEWithLogitsLoss()
optimizer = SGD(network.parameters(), lr=0.001)
losses = []
steps = 0
for epoch in range(epochs):
    start = True
    for data, labels in loader:
        optimizer.zero_grad()  # reset the parameter gradients
        results = network(data, drop_out)  # get predictions
        loss = loss_function(results, labels)  # estimate loss
        loss.backward()  # back-propagate gradients
        optimizer.step()  # update the parameter values (gradient-descent)
        losses.append(loss.data)  # keep track of the loss of this epoch
        if epoch % 5 == 0 and start:
            print('epoch {}, loss {}'.format(epoch, loss.data))
        start = False
        steps +=1

### Plot the results of Stochastic gradient descent optimization based learning from the data

In [ ]:
create_plots(steps, losses)